In [3]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
from sklearn.decomposition import PCA
import cv2
import os 
from skimage import filters
from scipy import ndimage

# Vamos a escribir un expression map, primero con us Self Organizing Map

In [4]:
b_shape = pd.read_csv('pca_shapepoints.csv',header = None)
b_shape

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,-1.921226,-2.226396,1.379754,0.684512,-0.760740,-0.074774,0.200652,-0.496513,0.491770,-0.700244,...,-0.350448,-0.010811,-0.053265,-0.060063,-0.219293,-0.024322,0.034714,-0.114054,-0.209845,0.021430
1,-1.856799,-2.227709,1.381573,0.674725,-0.756541,-0.090537,0.190973,-0.486900,0.487689,-0.699600,...,-0.351653,-0.013465,-0.049892,-0.051240,-0.215560,-0.029262,0.036563,-0.111783,-0.207176,0.030848
2,-1.856799,-2.227709,1.381573,0.674725,-0.756541,-0.090537,0.190973,-0.486900,0.487689,-0.699600,...,-0.351653,-0.013465,-0.049892,-0.051240,-0.215560,-0.029262,0.036563,-0.111783,-0.207176,0.030848
3,-1.966395,-2.225474,1.378478,0.691381,-0.763686,-0.063723,0.207430,-0.503255,0.494629,-0.700695,...,-0.349599,-0.008949,-0.055631,-0.066252,-0.221908,-0.020859,0.033418,-0.115646,-0.211716,0.014825
4,-2.585757,-2.212743,1.360827,0.786221,-0.804052,0.087823,0.300453,-0.595772,0.533952,-0.706846,...,-0.337827,0.016693,-0.087961,-0.151150,-0.257691,0.026667,0.015627,-0.137306,-0.237330,-0.076202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,5.499882,4.902797,-9.344429,-1.479545,2.271596,0.685373,1.983036,-3.248688,-1.682917,-0.898035,...,-0.414653,-0.288355,-0.134485,-0.256138,-0.132739,0.103614,0.049791,-0.358642,0.066591,0.088058
579,5.467658,4.921100,-9.246913,-1.387048,2.265947,0.694374,2.037743,-3.252593,-1.657882,-0.913983,...,-0.427157,-0.296115,-0.147364,-0.278768,-0.130351,0.126534,0.051857,-0.378555,0.074703,0.089757
580,5.501104,4.922888,-9.469544,-1.533058,2.298477,0.714105,1.968364,-3.285649,-1.745072,-0.846074,...,-0.434370,-0.281480,-0.090832,-0.247883,-0.141364,0.092260,0.047535,-0.356138,0.060861,0.095627
581,5.616209,4.926474,-9.517664,-1.607774,2.291651,0.661224,1.923488,-3.246323,-1.705099,-0.904727,...,-0.383642,-0.282397,-0.139666,-0.228141,-0.130089,0.075954,0.047965,-0.330734,0.059842,0.079986


In [6]:
b_structure = pd.read_csv('pca_structurepoints.csv',header = None)
b_cartoon = pd.read_csv('pca_cartoonpoints.csv',header = None)
b_texture = pd.read_csv('pca_texturepoints.csv',header = None)

In [14]:
b_structure

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-3.330332,2.548845,-5.386706,-0.460620,-0.971926,0.291840,1.901214,0.887607,2.116092,0.148901,-0.811308,-0.007965,0.062035
1,-3.330893,2.549205,-5.384598,-0.458234,-0.970186,0.290965,1.903637,0.893559,2.113924,0.151073,-0.812325,-0.007337,0.061448
2,-3.330893,2.549205,-5.384598,-0.458234,-0.970186,0.290965,1.903637,0.893559,2.113924,0.151073,-0.812325,-0.007337,0.061448
3,-3.329939,2.548592,-5.388185,-0.462294,-0.973146,0.292454,1.899514,0.883432,2.117612,0.147378,-0.810595,-0.008406,0.062446
4,-3.324535,2.545128,-5.408480,-0.485266,-0.989899,0.300877,1.876188,0.826134,2.138481,0.126468,-0.800807,-0.014451,0.068098
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,-3.746198,-1.514552,-0.549406,-1.684193,0.520818,-0.994905,0.095446,-0.707161,-0.084974,0.146269,0.260838,0.280381,-0.245087
150,-3.746052,-1.514646,-0.549955,-1.684814,0.520365,-0.994677,0.094814,-0.708712,-0.084409,0.145703,0.261103,0.280218,-0.244934
151,-3.746354,-1.514453,-0.548823,-1.683533,0.521299,-0.995147,0.096116,-0.705514,-0.085574,0.146870,0.260557,0.280555,-0.245249
152,-3.747210,-1.513904,-0.545608,-1.679893,0.523953,-0.996481,0.099811,-0.696437,-0.088880,0.150182,0.259006,0.281513,-0.246145


In [17]:
b_cartoon

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-205.255553,210.873360,4.982270,-91.401738,-111.396168,-59.082732,-69.710808,-25.336732,172.248394,23.799892,3.400180,5.286721,13.970702,-5.318015
1,-199.760933,219.516007,8.676643,-92.104047,-114.839393,-64.229713,-69.870853,-27.017541,184.414155,24.451272,9.588428,3.521479,17.485406,-6.703691
2,-199.696101,220.192376,4.758192,-93.952917,-117.933513,-67.839905,-69.041656,-28.988632,181.261442,25.345929,7.954378,-2.379163,18.127350,-7.733381
3,-202.700487,221.760162,5.975314,-90.716407,-114.647967,-64.741152,-66.820155,-23.998720,175.329636,23.370811,8.492309,6.115407,16.465881,-7.685741
4,-202.501186,225.191930,3.530385,-86.434451,-119.002391,-72.104096,-66.322129,-26.456183,162.753110,16.851668,7.981558,3.580255,13.276042,-5.013204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,498.777512,-11.206625,-51.738740,17.907682,-24.568076,3.468658,-2.526206,-3.256694,0.081984,-0.756964,7.495917,-14.076107,-40.382148,22.422566
150,499.892923,-13.036705,-52.616160,15.600512,-24.513089,-1.499128,-3.352787,-4.164315,0.039058,5.830351,8.726865,-13.104028,-39.055738,23.653262
151,497.461333,-15.401611,-54.794432,21.422197,-25.738143,-8.269544,-0.328639,-2.753618,0.181576,16.583259,13.525231,-10.708630,-40.191632,22.805027
152,498.688984,-18.568349,-55.960493,22.723150,-27.379433,-16.539628,1.107642,2.422395,3.047547,29.074966,21.062700,-8.918383,-35.585801,23.517306


In [10]:
b_shape = b_shape[0:154]
b_structure = b_structure[0:154]

In [18]:
b_vector = np.concatenate((b_shape, b_structure), axis = 1)
b_vector = np.concatenate((b_vector, b_cartoon), axis = 1)
b_vector = np.concatenate((b_vector, b_texture), axis = 1)

In [19]:
 b_vector.shape

(154, 86)

In [22]:
from minisom import MiniSom
### Initialization and training ###
som = MiniSom(7,7,86,sigma=1.0,learning_rate=0.5)
som.random_weights_init(b_vector)
print("Training...")
som.train_random(b_vector,100) # training with 100 iterations
print("\n...ready!")

Training...

...ready!
